In [ ]:
# Get articles :

In [12]:
import urllib.request
import io
import gzip
import os
from pathlib import Path
import pandas as pd
try:
    from Bio import Entrez
except ModuleNotFoundError:
    !pip install Bio
    from Bio import Entrez


response = urllib.request.urlopen('ftp://mirbase.org/pub/mirbase/CURRENT/miRNA.xls.gz')
compressed_file = io.BytesIO(response.read())
decompressed_file = gzip.GzipFile(fileobj=compressed_file)

with open(Path(os.getcwd(), 'miRNA.xlsx'), 'wb') as outfile:
    outfile.write(decompressed_file.read())

mir_database = pd.read_excel('miRNA.xlsx')

mir_database_1 = mir_database.loc[:, ['Accession', 'ID']]
mir_database_2 = mir_database.loc[:, ['Mature1_Acc', 'Mature1_ID']].rename(columns = {'Mature1_Acc':'Accession', 'Mature1_ID':'ID'})
mir_database_3 = mir_database.loc[:, ['Mature2_Acc', 'Mature2_ID']].rename(columns = {'Mature2_Acc':'Accession', 'Mature2_ID':'ID'})

final_database = pd.concat([mir_database_1, mir_database_2, mir_database_3])

In [3]:
def fetch_abstract(pmid):
    handle = Entrez.efetch(db='pubmed', id = pmid, retmode='xml')
    article = Entrez.read(handle)['PubmedArticle'][0]['MedlineCitation']['Article']
    if 'Abstract' in article:
            return article['Abstract']['AbstractText']
        
def concat_article(x):
    final_article = str()
    for i in range(len(x)):
        final_article = final_article + str(x[i]) + ' '
    return final_article

In [4]:
def get_literature(user_mir):
    
    filtered_database = final_database[final_database['Accession']  == user_mir]['ID']
    filtered_database = final_database[final_database['Accession']  == user_mir]['ID']

    if filtered_database.size == 1:
        mir = filtered_database.iloc[0]
        print('The accession number ' + user_mir + ' corresponds to miR ' + mir)
    else:
        print('miR accession is incorrect. Try again (caps sensitive)')

    Entrez.email = 'anonymous@gmail.com'
    esearch_query = Entrez.esearch(db="pubmed", term="mir-100", retmode="xml")
    esearch_result = Entrez.read(esearch_query)
    pmid_list = esearch_result['IdList']
    print("pmid's obtained: " + str(len(pmid_list)))
    
    abs_list = []

    for i in pmid_list:
        abs = fetch_abstract(i)
        abs_list.append(abs)
        
    abs_list = [concat_article(i) for i in abs_list if i is not None]
    
    return(abs_list)

In [26]:
training_mir = ['MI0000692', 'MI0000159', 'MI0000172', 'MI0000406', 'MI0000111']

In [27]:
all_abstracts = []

for i in training_mir:
    abstracts = get_literature(i)
    all_abstracts = all_abstracts + abstracts

The accession number MI0000692 corresponds to miR mmu-mir-100
pmid's obtained: 20
The accession number MI0000159 corresponds to miR mmu-mir-133a-1
pmid's obtained: 20
The accession number MI0000172 corresponds to miR mmu-mir-150
pmid's obtained: 20
The accession number MI0000406 corresponds to miR mmu-mir-106a
pmid's obtained: 20
The accession number MI0000111 corresponds to miR hsa-mir-105-1
pmid's obtained: 20


In [28]:
from nltk.tokenize import word_tokenize, sent_tokenize

all_sentences = []

for mir_abs in all_abstracts:
    abstr_sentences = sent_tokenize(mir_abs)
    all_sentences = all_sentences + abstr_sentences


In [29]:
all_sentences[10]

In [29]:
len(all_sentences)

1065

In [37]:
def make_training_data(string, pattern):
    counter = string.lower().find(pattern.lower())
    end_miR = len(string)
    
    while counter + 1 < len(string):
        counter += 1 
        if string[counter] in ".,!? :;":
            end_miR = counter
            counter = len(string)
            
    extracted_string = string[string.lower().find(pattern.lower()):end_miR]
    
    if string.lower().find(pattern.lower()) == -1 or not any(i.isdigit() for i in extracted_string):
        return((string, {'entities': []}))
    else:
        return((string, {'entities': [(string.lower().find(pattern.lower()), end_miR, 'miR')]}))


In [38]:
def check_training_data(string, pattern):
    counter = string.lower().find(pattern.lower())
    end_miR = len(string)
    
    while counter + 1 < len(string):
        counter += 1 
        if string[counter] in ".,!? :;":
            end_miR = counter
            counter = len(string)
           
    extracted_string = string[string.lower().find(pattern.lower()):end_miR]    
        
    if string.lower().find(pattern.lower()) == -1 or not any(i.isdigit() for i in extracted_string):
        return('')
    else:
        return(extracted_string)

In [32]:
all_sentences[0:5]

['MicroRNAs play critical roles in regulating target gene expression and multiple cellular processes in human cancer malignant progression.',
 'However, the function of miR-194 in gastric cancer (GC) remains unclear and controversial.',
 'In this study, we identified a series of miRNAs that can serve as prognostic biomarkers for GC by analysis of miRNA expression using TCGA (The Cancer Genome Atlas) data.',
 'Among them, miR-100, miR-125b, miR-199a and miR-194 were the 4 most promising prognostic biomarkers in GC due to their significant associations with various clinical characteristics of patients.',
 'MiR-100, miR-125b and miR-199a predicted poor prognosis in GC, while miR-194 predicted favorable prognosis in GC.']

In [39]:
print([make_training_data(i, 'miR') for i in all_sentences[0:5]])
print()
print([check_training_data(i, 'miR') for i in all_sentences[0:5]])

[('MicroRNAs play critical roles in regulating target gene expression and multiple cellular processes in human cancer malignant progression.', {'entities': []}), ('However, the function of miR-194 in gastric cancer (GC) remains unclear and controversial.', {'entities': [(25, 32, 'miR')]}), ('In this study, we identified a series of miRNAs that can serve as prognostic biomarkers for GC by analysis of miRNA expression using TCGA (The Cancer Genome Atlas) data.', {'entities': []}), ('Among them, miR-100, miR-125b, miR-199a and miR-194 were the 4 most promising prognostic biomarkers in GC due to their significant associations with various clinical characteristics of patients.', {'entities': [(12, 19, 'miR')]}), ('MiR-100, miR-125b and miR-199a predicted poor prognosis in GC, while miR-194 predicted favorable prognosis in GC.', {'entities': [(0, 7, 'miR')]})]

['', 'miR-194', '', 'miR-100', 'MiR-100']


In [40]:
training_data = [make_training_data(i, 'miR') for i in all_sentences]

In [43]:
training_data[1]

('However, the function of miR-194 in gastric cancer (GC) remains unclear and controversial.',
 {'entities': [(25, 32, 'miR')]})

In [49]:
import spacy
import random

In [63]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.7MB 4.4MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [64]:
nlp=spacy.load('en_core_web_sm')

# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [67]:
for _, annotations in training_data:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])
    
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [68]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 10 iterations
  for iteration in range(100):

    # shuufling examples  before every iteration
    random.shuffle(training_data)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

ValueError: [E989] `nlp.update()` was called with two positional arguments. This may be due to a backwards-incompatible change to the format of the training data in spaCy 3.0 onwards. The 'update' function should now be called with a batch of Example objects, instead of `(text, annotation)` tuples. 

In [52]:
test_miR = ['MI0000684']

In [57]:
all_test_abstracts = []

for i in test_miR:
    abstracts = get_literature(i)
    all_test_abstracts = all_test_abstracts + abstracts

all_test_sentences = []

for mir_abs in all_test_abstracts:
    abstr_sentences = sent_tokenize(mir_abs)
    all_test_sentences = all_test_sentences + abstr_sentences

The accession number MI0000684 corresponds to miR mmu-mir-107
pmid's obtained: 20


In [58]:
all_test_sentences[0:5]

['MicroRNAs play critical roles in regulating target gene expression and multiple cellular processes in human cancer malignant progression.',
 'However, the function of miR-194 in gastric cancer (GC) remains unclear and controversial.',
 'In this study, we identified a series of miRNAs that can serve as prognostic biomarkers for GC by analysis of miRNA expression using TCGA (The Cancer Genome Atlas) data.',
 'Among them, miR-100, miR-125b, miR-199a and miR-194 were the 4 most promising prognostic biomarkers in GC due to their significant associations with various clinical characteristics of patients.',
 'MiR-100, miR-125b and miR-199a predicted poor prognosis in GC, while miR-194 predicted favorable prognosis in GC.',
 'We also provide\xa0the\xa0first\xa0comprehensive transcriptome analysis of miR-194 in GC.',
 "Our data suggest that miR-194 tends to regulated target genes by binding to their 3' untranslated regions in a 7-mer-A1, 7-mer-m8 or 8-mer manner.",
 'KEGG pathway analysis sho

In [60]:
for doc in nlp.pipe(all_test_sentences[0:5]):
    # Print the document text and entitites
    print(doc.text)
    print(doc.ents, '\n')

MicroRNAs play critical roles in regulating target gene expression and multiple cellular processes in human cancer malignant progression.
() 

However, the function of miR-194 in gastric cancer (GC) remains unclear and controversial.
() 

In this study, we identified a series of miRNAs that can serve as prognostic biomarkers for GC by analysis of miRNA expression using TCGA (The Cancer Genome Atlas) data.
() 

Among them, miR-100, miR-125b, miR-199a and miR-194 were the 4 most promising prognostic biomarkers in GC due to their significant associations with various clinical characteristics of patients.
() 

MiR-100, miR-125b and miR-199a predicted poor prognosis in GC, while miR-194 predicted favorable prognosis in GC.
() 



In [ ]:
from spacy.lang.en import English
nlp = English()